In [1]:
# np.diff(x[1:],axis=0)

In [12]:
from dask.distributed import Client

In [13]:
client = Client('140.208.147.173:34671')

In [14]:
client

Client Scheduler: tcp://140.208.147.173:34671 Dashboard: http://140.208.147.173:8787/status,Cluster Workers: 1 Cores: 8 Memory: 30.00 GB


### Using Xarray and net Files

In [15]:
import xarray as xr
import cartopy
import cmocean
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.colorbar
import numpy

In [32]:
OM4p25.coords['xh']

<xarray.DataArray 'xh' (xh: 1440)>
array([-299.724244, -299.476198, -299.22815 , ...,   59.531631,   59.77967 ,
         60.027712])
Coordinates:
  * xh       (xh) float64 -299.7 -299.5 -299.2 -299.0 ... 59.53 59.78 60.03
Attributes:
    long_name:       h point nominal longitude
    units:           degrees_east
    cartesian_axis:  X

In [33]:
W25_av.coords['longitude'].shape

(1440,)

In [16]:
%%time

#300yr OM4files, date 198801 - 200712

#chunk along dim = 'time'
OM4p25 = xr.open_mfdataset('/work/Nkeh.Boh/SST/NetOMfiles/OM4p25/*.nc')
OM4p25z = xr.open_mfdataset('/work/Nkeh.Boh/SST/NetOMfiles/OM4p25z/*.nc')
OM4p5nep = xr.open_mfdataset('/work/Nkeh.Boh/SST/NetOMfiles/OM4p5nep/*.nc')
OM4p5mle =xr.open_mfdataset('/work/Nkeh.Boh/SST/NetOMfiles/OM4p5mle/*.nc')
OM4p5 = xr.open_mfdataset('/work/Nkeh.Boh/SST/NetOMfiles/OM4p5/*.nc')
# WOA files
W25 = xr.open_dataset('/work/Nkeh.Boh/SST/NetOMfiles/W25/WOA05_ptemp_salt_monthly.v20141007.nc')
W5 = xr.open_dataset('/work/Nkeh.Boh/SST/NetOMfiles/W05/WOA05_ptemp_salt_monthly.v2015.12.03.nc')
# 105GB < T_fileSize <110GB

#Grid files
G25 = xr.open_dataset('/work/Nkeh.Boh/SST/NetOMfiles/G25/ocean_hgrid.nc')
G5 = xr.open_dataset('/work/Nkeh.Boh/SST/NetOMfiles/G5/ocean_hgrid.nc')
M25 = xr.open_dataset('/work/Nkeh.Boh/SST/NetOMfiles/M25/ocean_mask.nc')
M5 = xr.open_dataset('/work/Nkeh.Boh/SST/NetOMfiles/M5/ocean_mask.nc')

OM4p25_av = OM4p25['tos'].mean(dim='time', skipna=True).load()
OM4p5nep_av = OM4p5nep['tos'].mean(dim='time', skipna=True).load()
OM4p25z_av = OM4p25z['tos'].mean(dim='time', skipna=True).load()
OM4p5_av = OM4p5['tos'].mean(dim='time', skipna=True).load()
OM4p5mle_av = OM4p5mle['tos'].mean(dim='time', skipna=True).load()

W25_av = W25['ptemp'].mean(axis=(0,1), skipna=True).load()
W5_av = W5['ptemp'].mean(axis=(0,1), skipna=True).load()

/nbhome/Nkeh.Boh/anaconda3/lib/python3.7/site-packages/xarray/core/nanops.py:159: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


CPU times: user 8.39 s, sys: 8.2 s, total: 16.6 s
Wall time: 3min 46s


In [ ]:
OM4p25_av.rename({'xh':'longitude','yh':'latitude'})
OM4p25_av['longitude'],OM4p25['latitude']= W25_av['longitude'], W25_av['latitude']

In [42]:
OM4p5nep_av.rename({'xh':'longititude','yh':'latitude'})
OM4p5nep_av['longitude'],OM4p5nep['latitude']= W5_av['longitude'], W5_av['latitude']

576

In [48]:
W5_av

<xarray.DataArray 'ptemp' (latitude: 576, longitude: 720)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)
Coordinates:
  * latitude   (latitude) float64 -77.91 -77.72 -77.54 ... 89.47 89.68 89.88
  * longitude  (longitude) float64 -299.8 -299.2 -298.8 ... 58.75 59.25 59.75

In [7]:
xq25 = G25.variables['x'][:][::2,::2]
yq25 = G25.variables['y'][:][::2,::2]
a25 = G25.variables['area'][:]; a25 = a25[::2,::2]+a25[1::2,1::2]+a25[1::2,::2]+a25[1::2,::2]
m25 = M25.variables['mask'][:]; a25 = a25*m25
xq5 = G5.variables['x'][:][::2,::2]
yq5 = G5.variables['y'][:][::2,::2]
a5 = G5.variables['area'][:]; a5 = a5[::2,::2]+a5[1::2,1::2]+a5[1::2,::2]+a5[1::2,::2]
m5 = M5.variables['mask'][:]; a5 = a5*m5

In [ ]:
def stats(ax, area, anomaly, label):
    mn = (anomaly*area).sum()/area.sum()
    sd = numpy.sqrt( ((anomaly-mn)**2*area).sum()/area.sum() )
    rms = numpy.sqrt( (anomaly**2*area).sum()/area.sum() )
    qmn, qmx = anomaly.min(), anomaly.max()
    print(label, 'mean =', mn, 'sd =', sd, 'rms =', rms, 'min =', qmn, 'max =', qmx )
    bb = ax.get_position()
    plt.gcf().text(bb.x0,bb.y1+.01,'mean=%.3f$^\circ$C'%mn, horizontalalignment='left')
    plt.gcf().text(bb.x1,bb.y1+.01,'rms=%.3f$^\circ$C'%rms, horizontalalignment='right')

In [8]:
%%time

fig = plt.figure(figsize=(12, 8))
vmin,vmax,ci,cmap = -2.25,2.25,.5,plt.cm.RdYlBu_r
axes = []
cilev = numpy.arange(vmin-ci,vmax+ci*2,ci)
norm = matplotlib.colors.BoundaryNorm(boundaries=cilev, ncolors=cmap.N)

q = OM4p25_av.rename({'xh':'','yh':}) - W25_av  #have to add 'tos' use WOA annual instead
ax = fig.add_subplot(2,2,1,projection=cartopy.crs.Robinson(central_longitude=-155))
im = ax.pcolormesh(xq25, yq25, q,
              transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, vmin=vmin, vmax=vmax)
stats(ax, a25, q, 'OMp25')
ax.coastlines()
axes.append(ax)
plt.title('a) OM4p25')

q = OM4p5nep_av - W5_av
ax = fig.add_subplot(2,2,2,projection=cartopy.crs.Robinson(central_longitude=-155))
im = ax.pcolormesh(xq5, yq5, q,
              transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
stats(ax, a5, q, 'OMp5nep')
ax.coastlines()
axes.append(ax)
plt.title('b) OM4p5n')

q = OM4p25z_av - W25_av
ax = fig.add_subplot(2,2,3,projection=cartopy.crs.Robinson(central_longitude=-155))
im = ax.pcolormesh(xq25, yq25, q,
              transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
stats(ax, a25, q, 'OMp25z')
ax.coastlines()
axes.append(ax)
plt.title('c) OM4p25z')

# q = OM4p5mle.variables['tos'][0,0]-W5.variables['ptemp'][0,0]
# ax = fig.add_subplot(2,2,3,projection=cartopy.crs.Robinson(central_longitude=-155))
# im = ax.pcolormesh(xq5, yq5, q,
#               transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
# stats(ax, a5, q, 'OM4p5mle')
# ax.coastlines()
# axes.append(ax)
# plt.title('c) OM4p25e')

q = OM4p5_av - W5_av
ax = fig.add_subplot(2,2,4,projection=cartopy.crs.Robinson(central_longitude=-155))
im = ax.pcolormesh(xq5, yq5, q,
              transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, vmin=vmin, vmax=vmax)
stats(ax, a5, q, 'OMp5')
ax.coastlines()
axes.append(ax)
plt.title('d) OM4p5')

ax = plt.gcf().add_axes((.25,.5,.5,.03))
cb = matplotlib.colorbar.ColorbarBase(ax=ax, cmap=cmap, norm=norm, boundaries=cilev,
                                      orientation='horizontal', extend='both')
cb.ax.set_title('SST bias ($^\circ$C)')

#plt.savefig('sst-bias.png')

MemoryError: 

<Figure size 864x576 with 0 Axes>